In [4]:
# Análise dos Top X Logradouros com Maior Média de Pessoas - VERSÃO SIMPLIFICADA V2 COM GRÁFICOS
# Base: Planilha PROCESSADA
# Usa: APENAS coluna "Logradouro" (sem campos parseados)
# Período: Solicitado ao usuário
# Top X: Quantidade personalizável

# %% [markdown]
# # 1. Configuração Inicial

# %%
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from pathlib import Path
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import matplotlib.dates as mdates

warnings.filterwarnings('ignore')

# Configurar estilo dos gráficos
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")

print("✓ Bibliotecas importadas")
print(f"✓ Análise iniciada em: {datetime.now().strftime('%d/%m/%Y %H:%M:%S')}")

# %% [markdown]
# # 2. Definir Período de Análise

# %%
print("\n" + "=" * 80)
print("DEFINIR PERÍODO DO RELATÓRIO")
print("=" * 80)

data_inicio_str = input("\nDigite a DATA INICIAL (dd/mm/aaaa): ")
data_fim_str = input("Digite a DATA FINAL (dd/mm/aaaa): ")

# Converter datas
try:
    data_inicio_dt = datetime.strptime(data_inicio_str, "%d/%m/%Y")
    data_fim_dt = datetime.strptime(data_fim_str, "%d/%m/%Y")
    
    # Converter para formato string usado nos filtros
    data_inicio = data_inicio_dt.strftime('%Y-%m-%d')
    data_fim = data_fim_dt.strftime('%Y-%m-%d')
    
    print(f"\n✓ Período selecionado: {data_inicio_dt.strftime('%d/%m/%Y')} a {data_fim_dt.strftime('%d/%m/%Y')}")
    
    # Calcular quantidade de dias
    dias_periodo = (data_fim_dt - data_inicio_dt).days + 1
    total_periodos = dias_periodo * 4  # 4 períodos por dia
    
    print(f"\n{'=' * 80}")
    print(f"Período: {data_inicio_dt.strftime('%d/%m/%Y')} a {data_fim_dt.strftime('%d/%m/%Y')}")
    print(f"  • Dias: {dias_periodo}")
    print(f"  • Períodos por dia: 4")
    print(f"  • Total de períodos: {total_periodos}")
    print(f"{'=' * 80}")
    
except ValueError:
    print("❌ Formato de data inválido! Use dd/mm/aaaa")
    raise Exception("Formato de data inválido")

# %% [markdown]
# # 3. Definir Quantidade de Logradouros (Top X)

# %%
print("\n" + "=" * 80)
print("DEFINIR QUANTIDADE DE LOGRADOUROS")
print("=" * 80)

try:
    top_x = int(input("\nDigite a quantidade de logradouros para análise (Top X): "))
    
    if top_x <= 0:
        print("❌ A quantidade deve ser maior que zero!")
        raise ValueError("Quantidade inválida")
    
    print(f"\n✓ Será analisado o Top {top_x} logradouros")
    print(f"{'=' * 80}")
    
except ValueError:
    print("❌ Valor inválido! Digite um número inteiro positivo.")
    raise Exception("Quantidade inválida")

# %% [markdown]
# # 4. Selecionar Arquivo Processado

# %%
from pathlib import Path

print("\n" + "=" * 80)
print("SELEÇÃO DO ARQUIVO PROCESSADO")
print("=" * 80)

# Detectar raiz do projeto
script_dir = Path(__file__).parent if '__file__' in globals() else Path.cwd()
if script_dir.name == 'notebooks':
    project_root = script_dir.parent
elif script_dir.name == 'etl':
    project_root = script_dir.parent.parent
else:
    project_root = script_dir

pasta_processed = project_root / 'data' / 'processed'
print(f"📂 Pasta processed: {pasta_processed}")

# Listar arquivos disponíveis
arquivos_disponiveis = sorted(list(pasta_processed.glob('*.xlsx')), 
                               key=lambda x: x.stat().st_mtime, 
                               reverse=True)

if arquivos_disponiveis:
    print(f"\n📁 Arquivos disponíveis (mais recentes primeiro):")
    for i, arq in enumerate(arquivos_disponiveis, 1):
        modificado = datetime.fromtimestamp(arq.stat().st_mtime).strftime('%d/%m/%Y %H:%M')
        print(f"  {i}. {arq.name}")
        print(f"     Modificado em: {modificado}\n")
    
    print("=" * 80)
    selecao = int(input("Digite o número do arquivo que deseja analisar: "))
    arquivo_selecionado = arquivos_disponiveis[selecao - 1]
    print(f"✓ Arquivo selecionado: {arquivo_selecionado.name}")
else:
    print(f"\n⚠️ Nenhum arquivo .xlsx encontrado em '{pasta_processed}'")
    print("   Execute primeiro: python src/etl/aplicar_parser_contagem_diaria.py")
    raise FileNotFoundError(f"Nenhum arquivo em {pasta_processed}")

print("=" * 80)

# %% [markdown]
# # 5. Carregar Dados Processados

# %%
print("\n" + "=" * 80)
print("CARREGANDO DADOS PROCESSADOS")
print("=" * 80)

try:
    df = pd.read_excel(arquivo_selecionado)
    print(f"✓ Arquivo carregado: {arquivo_selecionado.name}")
    print(f"✓ Total de registros: {len(df):,}")
    print(f"\nColunas disponíveis:")
    for col in df.columns:
        print(f"  - {col}")
    
    # Verificar campos necessários (APENAS os essenciais)
    campos_necessarios = ['Data', 'Logradouro', 'Período', 'Qtd. pessoas']
    campos_faltando = [c for c in campos_necessarios if c not in df.columns]
    
    if campos_faltando:
        print(f"\n✗ ERRO: Campos faltando: {campos_faltando}")
        raise KeyError("Campos necessários não encontrados")
    
except Exception as e:
    print(f"✗ ERRO ao carregar arquivo: {str(e)}")
    raise

# %% [markdown]
# # 6. Preparar Dados

# %%
print("\n" + "=" * 80)
print("PREPARANDO DADOS")
print("=" * 80)

# Converter data
df['Data'] = pd.to_datetime(df['Data'], errors='coerce')

# Garantir que Qtd. pessoas seja numérico
df['Qtd. pessoas'] = pd.to_numeric(df['Qtd. pessoas'], errors='coerce').fillna(0)

# Limpar espaços em branco no Logradouro e Período
df['Logradouro'] = df['Logradouro'].astype(str).str.strip()
df['Período'] = df['Período'].astype(str).str.strip()

print(f"✓ Campo 'Data' convertido para datetime")
print(f"✓ Campo 'Qtd. pessoas' convertido para numérico")
print(f"✓ Campo 'Logradouro' limpo")
print(f"✓ Registros preparados: {len(df):,}")

# Verificar períodos únicos
print(f"\nPeríodos encontrados:")
for periodo in sorted(df['Período'].unique()):
    print(f"  - {periodo}")

# %% [markdown]
# # 7. Filtrar Período

# %%
print("\n" + "=" * 80)
print("FILTRANDO PERÍODO")
print("=" * 80)

# Filtrar período
df_periodo = df[
    (df['Data'] >= data_inicio) & 
    (df['Data'] <= data_fim)
].copy()

print(f"Data início: {data_inicio}")
print(f"Data fim: {data_fim}")
print(f"Dias no período: {dias_periodo}")
print(f"✓ Registros no período: {len(df_periodo):,}")
print(f"  Logradouros únicos: {df_periodo['Logradouro'].nunique()}")
print(f"  Períodos únicos: {sorted(df_periodo['Período'].unique())}")

# %% [markdown]
# # 8. Agrupar e Calcular Médias

# %%
print("\n" + "=" * 80)
print("AGRUPANDO E CALCULANDO MÉDIAS (POR LOGRADOURO)")
print("=" * 80)

# Agrupar APENAS por Logradouro (sem separar por período)
df_agrupado = df_periodo.groupby('Logradouro').agg({
    'Qtd. pessoas': 'sum'
}).reset_index()

# Renomear coluna
df_agrupado.rename(columns={'Qtd. pessoas': 'Soma pessoas'}, inplace=True)

# Calcular Média pessoas (soma / total de períodos)
df_agrupado['Média pessoas'] = df_agrupado['Soma pessoas'] / total_periodos

# Ordenar por Média pessoas (decrescente)
df_agrupado = df_agrupado.sort_values('Média pessoas', ascending=False)

print(f"✓ Agrupamento concluído")
print(f"  Total de logradouros: {len(df_agrupado)}")
print(f"  Maior Média pessoas: {df_agrupado['Média pessoas'].max():.2f}")
print(f"  Menor Média pessoas: {df_agrupado['Média pessoas'].min():.2f}")
print(f"\n  Cálculo da média: Soma pessoas ÷ {total_periodos} ({dias_periodo} dias × 4 períodos)")

# %% [markdown]
# # 9. Selecionar Top X Média

# %%
print("\n" + "=" * 80)
print(f"SELECIONANDO TOP {top_x} MÉDIA")
print("=" * 80)

# Verificar se há logradouros suficientes
if len(df_agrupado) < top_x:
    print(f"⚠️ ATENÇÃO: Apenas {len(df_agrupado)} logradouros disponíveis.")
    print(f"   Ajustando Top X para {len(df_agrupado)}")
    top_x = len(df_agrupado)

# Pegar os X primeiros
df_topx = df_agrupado.head(top_x).copy()

print(f"✓ Top {top_x} Média selecionados")
print(f"\nPRÉVIA DO TOP {top_x} MÉDIA:")
print("=" * 80)

for idx, row in df_topx.iterrows():
    posicao = list(df_topx.index).index(idx) + 1
    print(f"{posicao}. {row['Logradouro']}")
    print(f"   Soma pessoas: {row['Soma pessoas']:,.0f}")
    print(f"   Média pessoas (por período): {row['Média pessoas']:.2f}")
    print()

# %% [markdown]
# # 10. Calcular Médias por Período

# %%
print("\n" + "=" * 80)
print("CALCULANDO MÉDIAS POR PERÍODO")
print("=" * 80)

# Mapear os nomes dos períodos
mapeamento_periodos = {
    '05h - Madrugada': 'Madrugada',
    '10h - Manhã': 'Manhã',
    '15h - Tarde': 'Tarde',
    '20h - Noite': 'Noite'
}

# Padronizar nomes dos períodos
df_periodo['Período_padrao'] = df_periodo['Período'].map(mapeamento_periodos)

# Para cada logradouro do Top X, calcular média por período
medias_por_periodo = []

for logradouro in df_topx['Logradouro']:
    df_log = df_periodo[df_periodo['Logradouro'] == logradouro].copy()
    
    # Agrupar por período e somar
    soma_por_periodo = df_log.groupby('Período_padrao')['Qtd. pessoas'].sum()
    
    # Calcular média (dividir pela quantidade de dias)
    media_madrugada = soma_por_periodo.get('Madrugada', 0) / dias_periodo
    media_manha = soma_por_periodo.get('Manhã', 0) / dias_periodo
    media_tarde = soma_por_periodo.get('Tarde', 0) / dias_periodo
    media_noite = soma_por_periodo.get('Noite', 0) / dias_periodo
    
    medias_por_periodo.append({
        'Logradouro': logradouro,
        'Média Madrugada': media_madrugada,
        'Média Manhã': media_manha,
        'Média Tarde': media_tarde,
        'Média Noite': media_noite
    })

df_medias_periodo = pd.DataFrame(medias_por_periodo)

# Merge com df_topx
df_topx_completo = df_topx.merge(df_medias_periodo, on='Logradouro')

print(f"✓ Médias por período calculadas para os Top {top_x}")

# %% [markdown]
# # 11. Criar Abas no Excel

# %%
print("\n" + "=" * 80)
print("CRIANDO ABAS NO EXCEL")
print("=" * 80)

# Caminho do arquivo de saída
pasta_docs = project_root / 'docs'
pasta_docs.mkdir(parents=True, exist_ok=True)
output_file = pasta_docs / f'top{top_x}_media_diaria_pessoas.xlsx'

# Criar o arquivo Excel
with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
    
    # Aba 1: Top X Média (com médias por período)
    df_topx_media_exportar = df_topx_completo[[
        'Logradouro', 
        'Soma pessoas', 
        'Média pessoas',
        'Média Madrugada',
        'Média Manhã',
        'Média Tarde',
        'Média Noite'
    ]].copy()
    df_topx_media_exportar.insert(0, '#', range(1, len(df_topx_media_exportar) + 1))
    df_topx_media_exportar.to_excel(writer, sheet_name=f'Top {top_x} Média', index=False)
    print(f"✓ Aba 'Top {top_x} Média' criada")
    
    # Criar uma aba para cada logradouro do Top X Média
    for idx, row in df_topx.iterrows():
        logradouro = row['Logradouro']
        
        # Filtrar dados originais para esse logradouro
        df_detalhado = df_periodo[
            df_periodo['Logradouro'] == logradouro
        ].copy()
        
        # Criar pivot: transformar períodos em colunas
        df_pivot = df_detalhado.pivot_table(
            index='Data',
            columns='Período',
            values='Qtd. pessoas',
            aggfunc='sum',
            fill_value=0
        ).reset_index()
        
        # Criar range completo de datas do período
        data_inicio_dt_range = pd.to_datetime(data_inicio)
        data_fim_dt_range = pd.to_datetime(data_fim)
        todas_datas = pd.date_range(start=data_inicio_dt_range, end=data_fim_dt_range, freq='D')
        
        # Criar DataFrame com todas as datas
        df_todas_datas = pd.DataFrame({'Data': todas_datas})
        
        # Fazer merge para incluir datas faltantes
        df_pivot = df_todas_datas.merge(df_pivot, on='Data', how='left')
        
        # Preencher valores faltantes com 0 (colunas de período)
        colunas_periodo = [col for col in df_pivot.columns if col != 'Data']
        df_pivot[colunas_periodo] = df_pivot[colunas_periodo].fillna(0)
        
        # Mapear os nomes dos períodos para os esperados
        mapeamento_periodos_export = {
            '05h - Madrugada': 'Madrugada',
            '10h - Manhã': 'Manhã',
            '15h - Tarde': 'Tarde',
            '20h - Noite': 'Noite'
        }
        
        # Renomear colunas de período
        df_pivot = df_pivot.rename(columns=mapeamento_periodos_export)
        
        # Garantir que todas as colunas de período existam
        for periodo in ['Madrugada', 'Manhã', 'Tarde', 'Noite']:
            if periodo not in df_pivot.columns:
                df_pivot[periodo] = 0
        
        # Preparar DataFrame para exportação na ordem correta
        df_exportar = pd.DataFrame()
        df_exportar['Logradouro'] = [logradouro] * len(df_pivot)
        df_exportar['Data'] = df_pivot['Data'].dt.strftime('%d/%m/%Y')
        df_exportar['Madrugada'] = df_pivot['Madrugada'].astype(int)
        df_exportar['Manhã'] = df_pivot['Manhã'].astype(int)
        df_exportar['Tarde'] = df_pivot['Tarde'].astype(int)
        df_exportar['Noite'] = df_pivot['Noite'].astype(int)
        
        # Nome da aba (limitado a 31 caracteres)
        posicao = list(df_topx.index).index(idx) + 1
        nome_aba = f"{posicao}. {logradouro[:25]}"
        if len(nome_aba) > 31:
            nome_aba = nome_aba[:31]
        
        # Escrever aba
        df_exportar.to_excel(writer, sheet_name=nome_aba, index=False)
        print(f"✓ Aba '{nome_aba}' criada ({len(df_exportar)} registros)")

print(f"\n✓ Arquivo criado: {output_file}")

# %% [markdown]
# # 12. Geração de Gráficos - Evolução Temporal

# %%
print("\n" + "=" * 80)
print("GERANDO GRÁFICOS DE EVOLUÇÃO TEMPORAL")
print("=" * 80)

# Criar pastas para salvar gráficos - NOVA ESTRUTURA
pasta_graficos = project_root / 'docs' / 'graficos'
pasta_regiao = pasta_graficos / 'elisios_stacecilia_staifigenia'
pasta_consolidado = pasta_regiao / 'consolidado'
pasta_graficos_individuais = pasta_regiao / 'logradouros_individuais'

pasta_graficos.mkdir(parents=True, exist_ok=True)
pasta_regiao.mkdir(parents=True, exist_ok=True)
pasta_consolidado.mkdir(parents=True, exist_ok=True)
pasta_graficos_individuais.mkdir(parents=True, exist_ok=True)

print(f"📁 Pasta principal: {pasta_graficos}")
print(f"📁 Pasta região: {pasta_regiao}")
print(f"📁 Pasta consolidado: {pasta_consolidado}")
print(f"📁 Pasta individuais: {pasta_graficos_individuais}")

# %%
def calcular_tendencia(datas, valores):
    """
    Calcula linha de tendência linear
    Retorna valores da tendência para plotagem e variação absoluta
    """
    # Converter datas para números (dias desde a primeira data)
    datas_num = [(d - datas.min()).days for d in datas]
    
    # Calcular regressão linear
    slope, intercept, r_value, p_value, std_err = stats.linregress(datas_num, valores)
    
    # Calcular valores da linha de tendência
    tendencia = [slope * x + intercept for x in datas_num]
    
    # Calcular variação ABSOLUTA do início ao fim (diferença no eixo Y)
    if len(tendencia) > 0:
        valor_inicio = tendencia[0]
        valor_fim = tendencia[-1]
        variacao_absoluta = valor_fim - valor_inicio
    else:
        variacao_absoluta = 0
    
    return tendencia, slope, r_value**2, variacao_absoluta

def limpar_nome_arquivo(nome):
    """
    Remove caracteres problemáticos para nomes de arquivos
    """
    # Caracteres problemáticos: : \ / ? * " < > |
    caracteres_problematicos = [':', '\\', '/', '?', '*', '"', '<', '>', '|']
    nome_limpo = nome
    for char in caracteres_problematicos:
        nome_limpo = nome_limpo.replace(char, '-')
    return nome_limpo

print("✓ Funções auxiliares definidas")

# %%
# CALCULAR VALOR MÁXIMO GLOBAL PARA PADRONIZAÇÃO DOS EIXOS
print("\n" + "=" * 80)
print("CALCULANDO VALOR MÁXIMO GLOBAL")
print("=" * 80)

# Encontrar o maior valor entre todos os logradouros (todos os períodos)
valor_max_global = 0

for logradouro in df_topx['Logradouro']:
    df_log = df_periodo[df_periodo['Logradouro'] == logradouro].copy()
    
    if len(df_log) > 0:
        max_logradouro = df_log['Qtd. pessoas'].max()
        
        if max_logradouro > valor_max_global:
            valor_max_global = max_logradouro

# Adicionar margem de 10% para melhor visualização
valor_max_global = int(valor_max_global * 1.1)

print(f"✓ Valor máximo global encontrado: {valor_max_global}")
print(f"  Este será o limite superior do eixo Y em todos os gráficos individuais")
print("=" * 80)

# %% [markdown]
# ## 12.1. Gráficos Individuais de Evolução por Logradouro

# %%
print("\n" + "=" * 80)
print("GERANDO GRÁFICOS INDIVIDUAIS POR LOGRADOURO")
print("=" * 80)

total_graficos = 0

# Paleta de cores - OPÇÃO 1 (Vibrante e Profissional)
cores_periodos = {
    'Madrugada': '#1B2A49',      # Azul escuro
    'Manhã': '#76B6FF',          # Azul claro
    'Tarde': '#E67E22',          # Laranja intenso
    'Noite': '#6C5CE7'           # Roxo escuro
}

cores_tendencia = {
    'Madrugada': '#1B2A49',      # Azul escuro
    'Manhã': '#76B6FF',          # Azul claro
    'Tarde': '#E67E22',          # Laranja intenso
    'Noite': '#6C5CE7'           # Roxo escuro
}

cores_rotulos = {
    'Madrugada': '#1B2A49',      # Azul escuro
    'Manhã': '#76B6FF',          # Azul claro
    'Tarde': '#E67E22',          # Laranja intenso
    'Noite': '#6C5CE7'           # Roxo escuro
}

for idx, row in df_topx.iterrows():
    logradouro = row['Logradouro']
    posicao = list(df_topx.index).index(idx) + 1
    
    print(f"\n[{posicao}/{top_x}] Processando: {logradouro[:60]}")
    
    # Filtrar dados do logradouro
    df_log = df_periodo[df_periodo['Logradouro'] == logradouro].copy()
    
    # Criar pivot com TODAS as datas do período
    data_inicio_dt_range = pd.to_datetime(data_inicio)
    data_fim_dt_range = pd.to_datetime(data_fim)
    todas_datas = pd.date_range(start=data_inicio_dt_range, end=data_fim_dt_range, freq='D')
    
    df_todas_datas = pd.DataFrame({'Data': todas_datas})
    
    df_pivot = df_log.pivot_table(
        index='Data',
        columns='Período_padrao',
        values='Qtd. pessoas',
        aggfunc='sum',
        fill_value=0
    ).reset_index()
    
    # Merge para incluir todas as datas
    df_pivot = df_todas_datas.merge(df_pivot, on='Data', how='left')
    
    # Preencher valores faltantes com 0
    for periodo in ['Madrugada', 'Manhã', 'Tarde', 'Noite']:
        if periodo not in df_pivot.columns:
            df_pivot[periodo] = 0
        else:
            df_pivot[periodo] = df_pivot[periodo].fillna(0)
    
    # ARREDONDAR OS DADOS ANTES DE PLOTAR
    df_pivot['Madrugada'] = df_pivot['Madrugada'].round(0).astype(int)
    df_pivot['Manhã'] = df_pivot['Manhã'].round(0).astype(int)
    df_pivot['Tarde'] = df_pivot['Tarde'].round(0).astype(int)
    df_pivot['Noite'] = df_pivot['Noite'].round(0).astype(int)
    
    # Criar o gráfico
    fig, ax = plt.subplots(figsize=(18, 10))
    
    # Marcadores diferentes para cada período
    marcadores = {'Madrugada': 'v', 'Manhã': 'o', 'Tarde': 's', 'Noite': '^'}
    
    # Plotar linhas de dados para cada período
    linhas = []
    for periodo in ['Madrugada', 'Manhã', 'Tarde', 'Noite']:
        linha = ax.plot(df_pivot['Data'], df_pivot[periodo], 
                marker=marcadores[periodo], linewidth=2.5, markersize=6, 
                label=periodo, color=cores_periodos[periodo], alpha=0.9, zorder=3)
        linhas.append(linha)
    
    # Adicionar rótulos de dados nas linhas
    offset_y = [10, 10, -15, -15]  # Offsets diferentes para cada período
    for i, periodo in enumerate(['Madrugada', 'Manhã', 'Tarde', 'Noite']):
        for data, valor in zip(df_pivot['Data'], df_pivot[periodo]):
            if valor > 0:
                ax.annotate(f'{valor}', 
                           xy=(data, valor), 
                           xytext=(0, offset_y[i]),
                           textcoords='offset points',
                           ha='center', va='bottom' if offset_y[i] > 0 else 'top',
                           fontsize=7, color=cores_rotulos[periodo],
                           fontweight='bold')
    
    # Calcular e plotar linhas de tendência
    tendencias_info = []
    for periodo in ['Madrugada', 'Manhã', 'Tarde', 'Noite']:
        tendencia, slope, r2, var_abs = calcular_tendencia(
            df_pivot['Data'], df_pivot[periodo]
        )
        
        sinal = '+' if var_abs >= 0 else ''
        
        ax.plot(df_pivot['Data'], tendencia, 
                linestyle='--', linewidth=2, 
                label=f'Tendência {periodo} ({sinal}{var_abs:.1f})', 
                color=cores_tendencia[periodo], alpha=0.7, zorder=2)
        
        tendencias_info.append(f'{periodo}: {sinal}{var_abs:.1f}')
    
    # PADRONIZAR EIXO Y
    ax.set_ylim(0, valor_max_global)
    
    # Configurar eixos
    ax.set_xlabel('Data', fontsize=13, fontweight='bold')
    ax.set_ylabel('Quantidade de Pessoas', fontsize=13, fontweight='bold')
    ax.set_title(
        f'Evolução - {logradouro}\n'
        f'Período: {data_inicio_dt.strftime("%d/%m/%Y")} a {data_fim_dt.strftime("%d/%m/%Y")}',
        fontsize=15, fontweight='bold', pad=20
    )
    
    # Formatar eixo X
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%d/%m'))
    ax.xaxis.set_major_locator(mdates.DayLocator(interval=max(1, dias_periodo // 15)))
    plt.xticks(rotation=45, ha='right')
    
    # Grid
    ax.grid(True, alpha=0.3, linestyle='--', zorder=1)
    
    # Legenda CENTRALIZADA ABAIXO
    ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.12), 
             ncol=4, fontsize=10, framealpha=0.9)
    
    # Estatísticas
    stats_text = (
        f'Madrugada: Média={int(df_pivot["Madrugada"].mean())} | '
        f'Max={int(df_pivot["Madrugada"].max())} | '
        f'Min={int(df_pivot["Madrugada"].min())}\n'
        f'Manhã: Média={int(df_pivot["Manhã"].mean())} | '
        f'Max={int(df_pivot["Manhã"].max())} | '
        f'Min={int(df_pivot["Manhã"].min())}\n'
        f'Tarde: Média={int(df_pivot["Tarde"].mean())} | '
        f'Max={int(df_pivot["Tarde"].max())} | '
        f'Min={int(df_pivot["Tarde"].min())}\n'
        f'Noite: Média={int(df_pivot["Noite"].mean())} | '
        f'Max={int(df_pivot["Noite"].max())} | '
        f'Min={int(df_pivot["Noite"].min())}'
    )
    ax.text(0.02, 0.98, stats_text, 
            transform=ax.transAxes, 
            fontsize=8, 
            verticalalignment='top',
            bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
    
    # Ajustar layout
    plt.tight_layout()
    plt.subplots_adjust(bottom=0.15)
    
    # Salvar
    nome_arquivo_limpo = limpar_nome_arquivo(logradouro[:50])
    nome_arquivo = f"{posicao:02d}_{nome_arquivo_limpo}.png"
    caminho_arquivo = pasta_graficos_individuais / nome_arquivo
    plt.savefig(caminho_arquivo, dpi=300, bbox_inches='tight')
    plt.close()
    
    total_graficos += 1
    print(f"  ✓ Gráfico salvo")

print(f"\n{'=' * 80}")
print(f"✓ Total de gráficos individuais gerados: {total_graficos}")
print(f"{'=' * 80}")

# %% [markdown]
# ## 12.2. Gráfico Consolidado - Visão Geral

# %%
print("\n" + "=" * 80)
print("GERANDO GRÁFICO CONSOLIDADO - VISÃO GERAL")
print("=" * 80)

# Preparar dados
df_topx_plot = df_topx_completo.copy()

# ARREDONDAR OS DADOS
df_topx_plot['Média Madrugada'] = df_topx_plot['Média Madrugada'].round(0).astype(int)
df_topx_plot['Média Manhã'] = df_topx_plot['Média Manhã'].round(0).astype(int)
df_topx_plot['Média Tarde'] = df_topx_plot['Média Tarde'].round(0).astype(int)
df_topx_plot['Média Noite'] = df_topx_plot['Média Noite'].round(0).astype(int)

# Criar figura
fig, ax = plt.subplots(figsize=(18, max(12, top_x * 0.6)))

# Preparar dados
x = np.arange(len(df_topx_plot))
width = 0.2

# Criar barras agrupadas
bars1 = ax.barh(x - 1.5*width, df_topx_plot['Média Madrugada'], width,
                label='Madrugada', color=cores_periodos['Madrugada'], alpha=0.9)
bars2 = ax.barh(x - 0.5*width, df_topx_plot['Média Manhã'], width,
                label='Manhã', color=cores_periodos['Manhã'], alpha=0.9)
bars3 = ax.barh(x + 0.5*width, df_topx_plot['Média Tarde'], width,
                label='Tarde', color=cores_periodos['Tarde'], alpha=0.9)
bars4 = ax.barh(x + 1.5*width, df_topx_plot['Média Noite'], width,
                label='Noite', color=cores_periodos['Noite'], alpha=0.9)

# Configurar
ax.set_yticks(x)
ax.set_yticklabels(df_topx_plot['Logradouro'], fontsize=10)
ax.set_xlabel('Média de Pessoas por Período', fontsize=13, fontweight='bold')
ax.set_title(
    f'Top {top_x} Logradouros - Média de Pessoas por Período (Consolidado)\n'
    f'Período: {data_inicio_dt.strftime("%d/%m/%Y")} a {data_fim_dt.strftime("%d/%m/%Y")} '
    f'({dias_periodo} dias, {total_periodos} períodos)',
    fontsize=15, fontweight='bold', pad=20
)

# Rótulos
for bars in [bars1, bars2, bars3, bars4]:
    for bar in bars:
        width_val = bar.get_width()
        if width_val > 0:
            ax.text(width_val, bar.get_y() + bar.get_height()/2,
                   f' {int(width_val)}',
                   va='center', fontsize=8, fontweight='bold')

# Inverter eixo Y
ax.invert_yaxis()

# Grid
ax.grid(axis='x', alpha=0.3, linestyle='--')

# Legenda
ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.06), 
         ncol=4, fontsize=11, framealpha=0.9)

# Layout
plt.tight_layout()
plt.subplots_adjust(bottom=0.08)

# Salvar
caminho_consolidado = pasta_consolidado / f'top{top_x}_consolidado_periodos.png'
plt.savefig(caminho_consolidado, dpi=300, bbox_inches='tight')
plt.close()

print(f"✓ Gráfico consolidado salvo: top{top_x}_consolidado_periodos.png")
print(f"✓ Localização: {pasta_consolidado}")
print(f"{'=' * 80}")

# %% [markdown]
# ## 12.3. Resumo dos Gráficos Gerados

# %%
print("\n" + "=" * 80)
print("RESUMO - GRÁFICOS GERADOS")
print("=" * 80)

total_individuais = len(list(pasta_graficos_individuais.glob('*.png')))
total_consolidado = len(list(pasta_consolidado.glob('*.png')))

print(f"""
📊 GRÁFICOS DE EVOLUÇÃO TEMPORAL

ESTRUTURA DE PASTAS:
  📁 graficos/
      - elisios_stacecilia_staifigenia/
          - consolidado/
              - top{top_x}_consolidado_periodos.png
          - logradouros_individuais/

TOTAIS GERADOS:
  • Gráficos individuais (evolução temporal): {total_individuais}
    - Linhas de evolução diária (4 períodos)
    - Linhas de tendência com variação absoluta no eixo Y
    - Estatísticas (média, máx, mín) por período
    - Eixos padronizados (0 a {valor_max_global})
    - Rótulos com valores inteiros
  
  • Gráfico consolidado: {total_consolidado}
    - Top {top_x} geral com médias por período
    - Nomes completos dos logradouros
    - Rótulos com valores inteiros

TOTAL: {total_individuais + total_consolidado} gráficos

CORES DOS PERÍODOS (Paleta Vibrante e Profissional):
  Madrugada: Roxo vibrante
  Manhã: Vermelho forte
  Tarde: Azul royal
  Noite: Laranja dourado

CARACTERÍSTICAS:
  ✓ Resolução: 300 DPI (alta qualidade)
  ✓ Período analisado: {data_inicio_dt.strftime('%d/%m/%Y')} a {data_fim_dt.strftime('%d/%m/%Y')}
  ✓ Total de dias: {dias_periodo}
  ✓ Total de períodos: {total_periodos}
  ✓ Linhas de tendência com variação absoluta
  ✓ Estatísticas descritivas por período
  ✓ Eixos Y padronizados
  ✓ Valores inteiros em rótulos
  ✓ Legendas centralizadas
  ✓ Nomes completos dos logradouros
  ✓ Cores vibrantes e contrastantes

""")
print("=" * 80)
print("✓ GERAÇÃO DE GRÁFICOS CONCLUÍDA!")
print(f"✓ {datetime.now().strftime('%d/%m/%Y %H:%M:%S')}")
print("=" * 80)

✓ Bibliotecas importadas
✓ Análise iniciada em: 11/11/2025 12:30:16

DEFINIR PERÍODO DO RELATÓRIO

✓ Período selecionado: 01/10/2025 a 31/10/2025

Período: 01/10/2025 a 31/10/2025
  • Dias: 31
  • Períodos por dia: 4
  • Total de períodos: 124

DEFINIR QUANTIDADE DE LOGRADOUROS

✓ Será analisado o Top 10 logradouros

SELEÇÃO DO ARQUIVO PROCESSADO
📂 Pasta processed: c:\Users\x504693\Documents\projetos\projeto_etl_dados\data\processed

📁 Arquivos disponíveis (mais recentes primeiro):
  1. Contagem_diaria_centro - Padronizada_processada_20251110_185845.xlsx
     Modificado em: 10/11/2025 18:59

  2. Contagem_diaria_centro - Padronizada_processada_20251107_180111.xlsx
     Modificado em: 07/11/2025 18:01

  3. lista_logradouros_corretos.xlsx
     Modificado em: 04/11/2025 11:08

  4. cruzamento_nomes_base_okuhara.xlsx
     Modificado em: 21/10/2025 18:58

  5. cruzamento_nomes_base_tablet.xlsx
     Modificado em: 21/10/2025 18:51

✓ Arquivo selecionado: Contagem_diaria_centro - Padronizada